In [1]:
# Import Libraries

import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf 

# Import Neural Net & Data Libraries

from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, BatchNormalization
import keras.utils
import keras.metrics



In [5]:
# Load Data
data = keras.utils.image_dataset_from_directory('data', labels='inferred', label_mode='int')
scaled_data = data.map(lambda x,y: (x/255, y))


Found 12619 files belonging to 2 classes.


In [11]:
from keras.preprocessing.image import ImageDataGenerator
import os
# datagen = ImageDataGenerator(
#     rotation_range=30, 
#     fill_mode='nearest'
# )

# aug_iter = datagen.flow(data[0][0], batch_size=1)
# fig, ax = plt.subplots(nrows=1, ncols=3, figsize = (15,15))

# for i in range(5):
#     image = next(aug_iter)[0].astype(int)

#     ax[i] = plt.imshow(image)
#     plt.show()

In [ ]:
# Assign Data

test_len = int(0.1 * len(scaled_data))
train_len = int(0.6 * len(scaled_data))
val_len = int(0.3 * len(scaled_data))

test = scaled_data.take(test_len)
train = scaled_data.skip(test_len).take(train_len)
valid = scaled_data.skip(train_len + test_len).take(val_len)

In [ ]:
# Model Creation

model = Sequential([

Conv2D(64, (3,3),1, activation='relu', input_shape=(256,256,3)),
MaxPooling2D(), 
Dropout(0.25),
 
Conv2D(32, (3,3),1, activation='relu'),
MaxPooling2D(),
Dropout(0.25),

Conv2D(64, (3,3),1, activation='relu'),
MaxPooling2D(),
Dropout(0.25),

Flatten(),

Dense(256, activation='relu'),
Dropout(0.3),

Dense(1, activation='sigmoid')







    
])

In [ ]:
model.compile(optimizer='adam', loss = tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
log_directory = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_directory)

model.fit(train, epochs=10, validation_data=valid, callbacks=[tensorboard_callback])

In [ ]:
recall = keras.metrics.Recall()
accuracy = keras.metrics.BinaryAccuracy()
precision = keras.metrics.Precision()

for batch in test.as_numpy_iterator():
    X, y = batch

    yhat = model.predict(X)

    recall.update_state(y, yhat)
    accuracy.update_state(y, yhat)
    precision.update_state(y, yhat)


In [ ]:
print(f"Recall:{recall.result().numpy()}, Accuracy:{accuracy.result().numpy()}, Precision:{precision.result().numpy()}")

In [ ]:
import os

In [ ]:
# first_model = load_model(os.path.join('models','model.h5'))

In [ ]:
image = cv2.imread(os.path.join('test_images','test4n.jpg'))
resize = tf.image.resize(image, (256,256))

plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims((resize/255), 0))

In [ ]:
if yhat >= 0.5:
    print("This person is wearing a mask")
elif yhat < 0.5:
    print("This person is not wearing a mask")

In [ ]:
yhat

In [ ]:
model.save(os.path.join('models','model.h5'))